In [ ]:
# Import libraries
import pandas as pd
from datetime import datetime
import glob

In [ ]:
# Merge CSVs
file_paths = glob.glob('compras_*.csv')
dfs = [pd.read_csv(file, sep=';') for file in file_paths]
df = pd.concat(dfs, ignore_index=True)

# Save resulting DataFrame to a purchases.csv file
df.to_csv('compras.csv', index=False)

In [ ]:
# Convert creation_date column to YYYY-MM-DD format
df['creation_date'] = pd.to_datetime(df['creation_date']).dt.strftime('%Y-%m-%d')

# Sort DataFrame by account and period
df.sort_values(by=['account', 'period'], inplace=True)

In [ ]:
# Function to calculate requested variables per row
def calcular_variables_compras(row):
    cliente = df[df['account'] == row['account']]
    periodo_actual = row['period']
    ultimos_3_meses = cliente[(cliente['period'] >= periodo_actual - 300) & (cliente['period'] < periodo_actual)]
    
    # Set to exclude the current period if there is not enough historical data
    if ultimos_3_meses['period'].nunique() < 3:
        ultimos_3_meses = cliente[(cliente['period'] >= periodo_actual - 200) & (cliente['period'] < periodo_actual)]
    
    row['compras_u3m'] = int(ultimos_3_meses['buy'].any())
    row['monto_compras_u3m'] = ultimos_3_meses['amount'].sum()
    row['veces_compras_u3m'] = ultimos_3_meses['times'].sum()
    
    if ultimos_3_meses.empty:
        row['periodo_ultima_compra'] = 0
        row['meses_sin_comprar'] = periodo_actual % 100
    else:
        row['periodo_ultima_compra'] = ultimos_3_meses['period'].iloc[-1]
        row['meses_sin_comprar'] = (periodo_actual - row['periodo_ultima_compra']) % 100
    
    return row

In [ ]:
# Apply function to each row of the DataFrame
df = df.apply(calcular_variables_compras, axis=1)

# PLUS: Save resulting DataFrame (with news columns) to a purchases.csv file
df.to_csv('purchases.csv', index=False)